In [1]:
import pyspark
print(pyspark.__version__)

3.0.0


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
spark = SparkSession \
    .builder \
    .appName("Wrangling sf-crime Data") \
    .getOrCreate()

In [6]:
path = "./police-department-calls-for-service.json"
df = spark.read.json(path,multiLine=True)

In [7]:
df.printSchema()

root
 |-- address: string (nullable = true)
 |-- address_type: string (nullable = true)
 |-- agency_id: string (nullable = true)
 |-- call_date: timestamp (nullable = true)
 |-- call_date_time: timestamp (nullable = true)
 |-- call_time: string (nullable = true)
 |-- city: string (nullable = true)
 |-- common_location: string (nullable = true)
 |-- crime_id: string (nullable = true)
 |-- disposition: string (nullable = true)
 |-- offense_date: timestamp (nullable = true)
 |-- original_crime_type_name: string (nullable = true)
 |-- report_date: timestamp (nullable = true)
 |-- state: string (nullable = true)



In [13]:
# TODO select original_crime_type_name and disposition
distinct_table = df.select("call_date_time","original_crime_type_name","disposition").distinct().withWatermark("call_date_time","10 minutes")
distinct_table.show()

+-------------------+------------------------+------------+
|     call_date_time|original_crime_type_name| disposition|
+-------------------+------------------------+------------+
|2018-12-31 22:29:00|                   Caser|         REP|
|2018-12-31 20:41:00|                    Npat|         GOA|
|2018-12-31 18:27:00|            Passing Call|Not recorded|
|2018-12-31 16:05:00|                     601|         GOA|
|2018-12-31 15:08:00|       Assault / Battery|         ABA|
|2018-12-31 02:42:00|           Audible Alarm|         PAS|
|2018-12-30 16:58:00|            Passing Call|Not recorded|
|2018-12-30 15:54:00|                   Drugs|         ADV|
|2018-12-30 15:22:00|            Passing Call|Not recorded|
|2018-12-30 14:07:00|         Fight No Weapon|          ND|
|2018-12-30 12:36:00|          Meet W/citizen|         HAN|
|2018-12-30 11:33:00|      Mentally Disturbed|         GOA|
|2018-12-30 09:03:00|      Auto Boost / Strip|          ND|
|2018-12-30 08:31:00|     Traf Violation

In [15]:
# count the number of original crime type
agg_df = distinct_table.groupBy("original_crime_type_name").count().orderBy("count", ascending=False)
agg_df.show()

+------------------------+-----+
|original_crime_type_name|count|
+------------------------+-----+
|            Passing Call|32123|
|            Traffic Stop|13041|
|     Traf Violation Cite| 9992|
|       Suspicious Person| 9910|
|      Homeless Complaint| 5950|
|              Trespasser| 5799|
|           Audible Alarm| 5727|
|                  22500e| 5617|
|        Well Being Check| 5514|
|         Muni Inspection| 5159|
|      Suspicious Vehicle| 5002|
|         Fight No Weapon| 4185|
|          Noise Nuisance| 4052|
|      Auto Boost / Strip| 2951|
|      Traf Violation Tow| 2865|
|                    Poss| 2847|
|      Mentally Disturbed| 2576|
|             Petty Theft| 2540|
|          Meet W/citizen| 2399|
|       Assault / Battery| 2352|
+------------------------+-----+
only showing top 20 rows

